# Metadata

```yaml
course:   DS 5001 
module:   Final Project
topic:    Exploratory Text Analytics using Television Sitcom Scripts Final Report
author:   Eric Tria
date:     5 May 2023
```

# Student Info

```yaml
name:     Eric Tria
user_id:  emt4wf
email:    emt4wf@virginia.edu
```

# Project: Exploratory Text Analytics using Television Sitcom Scripts

# 1. Introduction

Television shows have been around for decades and have been part of the daily routine of a lot of people. One interesting thing to notice is how shows have somewhat changed over the years. For this project, I will be focusing on sitcoms or situational comedies. In earlier years, sitcoms were known for being filmed in front of a live audience where the laughter and reactions are recorded. More recently, however, single-camera comedies are becoming more popular. Single-camera comedies are filmed without a live audience where the scripts are more similar a [feature comedy](https://screencraft.org/blog/differences-single-camera-multi-camera-tv-pilot-scripts/). Due to these differences, I will be focusing on epsisode scripts of three popular single-camera sitcoms from the 2000s and 2010s: Parks and Recreation (2009-2015), Brooklyn Nine-Nine (2013-2021), and The Office US (2005-2013). I selected these three shows because they revolve around characters in various workspaces: a government office, police precinct, and corporate office. Given these three popular shows, are there similarities between their scripts in terms of the general topics, language used, sentiments, and such? To answer this question, I went through the scripts of all the episodes of these shows. 

The corpus used for this project contains episode scripts for all the seasons of each show: 7 seasons for Parks and Recreation, 8 for Brooklyn Nine-Nine, and 9 for The Office US. A sitcom episode usually runs between 20 to 30 minutes, so each episode script would be relatively short compared to a full movie. Sitcoms usually follow a three-act structure [source](https://www.masterclass.com/articles/how-to-write-a-sitcom) where each act consists of 3-5 scenes each [source](https://www.theatlantic.com/entertainment/archive/2014/12/cracking-the-sitcom-code/384068/). In total, this provided me with a large corpus to work with in order to extract insights that can possibly answer the initial question that I posed.

# 2. Data

### 2.1 Provenance

The episode scripts were programmatically scraped from [Sublikescript.com](https://subslikescript.com/), which is a website that contains scripts of a wide range of shows. The website had the scripts for the shows that I wanted to use for my project. The links I used were:
- [Parks and Recreation](https://subslikescript.com/series/Parks_and_Recreation-1266020)
- [Brooklyn Nine-Nine](https://subslikescript.com/series/Brooklyn_Nine-Nine-2467372)
- [The Office US](https://subslikescript.com/series/The_Office-386676) 

In order to do this, I wrote a Python class to do the web scraping which is located in `/lib/script_scraper.py`. This Python class takes in a Sublikescript link and converts it into a corpus of lines and tokens. 

In addition to the scripts, I extracted additional data from Wikipedia to add more data to each episode. I was able to eaxtract information such as episode title, director, writer, date, and the total US viewers in millions for each episode. For each season, I was also able to get the Rotten Tomato rating, which can show how critics view these shows. I also wrote a Python script to scrape the data from Wikipedia located in `/lib/wiki_scraper.py`, which takes a link and converts the details into a tabular format. This information was extracted from the following links:
- Parks and Recreation: [Wikipedia page](https://en.wikipedia.org/wiki/Parks_and_Recreation) and [episode list](https://en.wikipedia.org/wiki/List_of_Parks_and_Recreation_episodes)
- Brooklyn Nine-Nine: [Wikipedia page](https://en.wikipedia.org/wiki/Brooklyn_Nine-Nine) and [episode list](https://en.wikipedia.org/wiki/List_of_Brooklyn_Nine-Nine_episodes)
- The Office US: [Wikipedia page](https://en.wikipedia.org/wiki/The_Office_(American_TV_series) and [episode list](https://en.wikipedia.org/wiki/List_of_The_Office_(American_TV_series)_episodes)

The process of using these scripts to generate the corpus is available in a Jupyter notebook called `EXTRACT_DATA.ipynb`.

### 2.2 Location

The raw scraped data can be accessed through at the `/raw` folder of the repository. The different files are:
- Parks and Recreation:
    - `/raw/PNR-LINES.csv`
    - `/raw/PNR-TOKENS.csv`
    - `/raw/PNR-LIB.csv`
    - `/raw/PNR-LIB-EPS.csv`
- Brooklyn Nine-Nine:
    - `/raw/B99-LINES.csv`
    - `/raw/B99-TOKENS.csv`
    - `/raw/B99-LIB.csv`
    - `/raw/B99-LIB-EPS.csv`
- The Office US:
    - `/raw/OFFICE-LINES.csv`
    - `/raw/OFFICE-TOKENS.csv`
    - `/raw/OFFICE-LIB.csv`
    - `/raw/OFFICE-LIB-EPS.csv`

I have also provided files that combined all three series in the `/final` folder of the repository. The different files are:
- `/final/ALL-CORPUS.csv`
- `/final/ALL-LIB.csv`
- `/final/ALL-LIB-EPS.csv`

### 2.3 Description

In general, the data would be split in the following: series, seasons, episodes, scenes, lines, and tokens. Since the data source did not have explicit scene tagging, I used the structure discusses earlier where each sitcom would have 3 acts with 5 scenes each. For this project, I used an arbitrary number of 15 scenes per episode.

#### Files under `/raw`
The raw version of the corpus called `LINES` contains lines for each script. This table is formatted as follows:

| Column     | Type    | Description          |
|------------|---------|----------------------|
| series_id  | Index   | ID for the TV series |
| season_id  | Index   | Season number        |
| episode_id | Index   | Episode number       |
| scene_id   | Index   | Scene number         |
| line_id    | Index   | Line number          |
| line       | Feature | Line contents        |

The tokenized version of the corpus called `TOKENS` contains the script broken down into individual tokens. This table is formatted as follows:

| Column     | Type    | Description          |
|------------|---------|----------------------|
| series_id  | Index   | ID for the TV series |
| season_id  | Index   | Season number        |
| episode_id | Index   | Episode number       |
| scene_id   | Index   | Scene number         |
| line_id    | Index   | Line number          |
| token_id   | Index   | Token number         |
| token_str  | Feature | String token         |
| term_str   | Feature | Cleaned string term  |

The `LIB` tables contain additional information on a season level:

| Column           | Type    | Description                              |
|------------------|---------|------------------------------------------|
| series_id        | Index   | ID for the TV series                     |
| season_id        | Index   | Season number                            |
| num_episodes     | Feature | Number of episodes in the season         |
| year             | Feature | TV season year of the season             |
| viewers_millions | Feature | Average number of US viewers in millions |
| rt_rating        | Feature | Rotten Tomato rating                     |
| series_name      | Feature | Series name                              |

The `LIB-EPS` tables contain additional information on an episode level:

| Column        | Type    | Description                       |
|---------------|---------|-----------------------------------|
| series_id     | Index   | ID for the TV series              |
| season_id     | Index   | Season number                     |
| episode_id    | Index   | Episode number                    |
| episode_title | Feature | Title of the episode              |
| director      | Feature | Name of the episode's director    |
| first_writer  | Feature | Name of the epsiode's main writer |
| date          | Feature | Date when the episode aired       |
| us_viewers    | Feature | Number of US viewers in millions  |

#### Files under `/final`

The `ALL-CORPUS.csv` file contains the same details as the raw `TOKENS` files but with additional features:

| Column     | Type    | Description          |
|------------|---------|----------------------|
| pos_tuple  | Feature   | Token and its tagged part of speech |
| pos        | Feature   | Tagged part of speech       |

The `ALL-LIB.csv` file contains the same details as the raw `LIB` files but with additional engineered features:

| Column     | Type    | Description          |
|------------|---------|----------------------|
| series_season  | Feature   | Combined label using series name and season number |
| series_rotten_tomatoes        | Feature   | Category if rating is 90+ or not      |

The `ALL-LIB-EPS.csv` file contains the same details as the raw `LIB-EPS` files but with additional engineered features:

| Column     | Type    | Description          |
|------------|---------|----------------------|
| series_season_ep  | Feature   | Combined label using series name, season number, and episode number |
| year        | Feature   | Year when the episode aired      |

In terms of sizes, the breakdown is as follows:
- `ALL-CORPUS` has a total of **1,563,801** tokens
- `ALL-LIB` has a total of **24** seasons
- `ALL-LIB-EPS` has a total of **479** episodes

### 2.4 Format

All of the files listed above are in CSV format. The sources, however, were initially in HTML format since they were scraped from web pages. The HTML was transformed into text using BeautifulSoup's LXML parser and then that text was converted into the CSV files.

# 3. Models

Using this source data, I proceeded to create additional models to aid in my analysis of the corpus. These models and algorithms included the following: term frequency, hierarchical clustering, principal component analysis, topic modelling, word embeddings, and sentiment analysis.

### 3.0 OHCO Explanation

The OHCO or Ordered Hierarchy of Content Objects that I used for this corpus is:
- Series
- Seasons
- Episodes
- Scenes
- Lines
- Tokens

For my analysis, I will treat this OHCO levels similarly to the usual OHCO levels for books:
- Seasons used similarly to Books
- Episodes used similarly to Chapters
- Scenes used similarly to Paragraphs
- Lines used similarly to Sentences

### 3.1 VOCAB / Term Frequency

The first model that I created is using term frequency and other metrics derived from it. This makes up the `VOCAB` table for my corpus. This table highlights the TFIDF or Term Frequency - Inverse Document Frequency, which is the frequency of a word in a document weighted by that word's information value over the corpus
. The summary of the included fields are as follows:

| Column       | Type    | Description                                                 |
|--------------|---------|-------------------------------------------------------------|
| term_str     | Index   | String term                                                 |
| n            | Feature | Term frequency                                              |
| n_chars      | Feature | Term length                                                 |
| p            | Feature | Term probability                                            |
| i            | Feature | Term information                                            |
| h            | Feature | Term entropy                                                |
| max_pos      | Feature | Most frequently associated part-of-speech character         |
| n_pos        | Feature | POS ambiguity                                               |
| cat_pos      | Feature | POS ambiguity                                               |
| stop         | Feature | Dummy encoded variable if term is part of NLTK's stop words |
| tfidf_mean   | Feature | Mean value of term TFIDF                                    |
| tfidf_median | Feature | Median value of term TFIDF                                  |
| tfidf_max    | Feature | Max value of term TFIDF                                     |
| dfidf        | Feature | Document frequency                                          |

For the whole corpus, TFIDF and DFIDF values were computed using **EPISODES** as the OHCO level due to memory constraints. This file is located at:
- `/final/ALL-VOCAB-EPS.csv`

Additionally, I created separate versions of the VOCAB table for each TV series with the TFIDF and DFIDF values computed using **SCENES** as the OHCO level. These files are located at:
- `/final/PNR-VOCAB-SCENES.csv`
- `/final/B99-VOCAB-SCENES.csv`
- `/final/B99-VOCAB-SCENES.csv`

I created this VOCAB tables using the functions from the `lib/corpus_enhancer.py` Python class and demonstrated in `EXTRACT_DATA.ipynb`.

### 3.2 Hierarchical Clustering

To create the hierarchical clustering model, I used the 1000 most significant terms in the VOCAB table using DFIDF as the significance measure. I also filtered the terms to those whose maximum part-of-speech belong to `NN NNS VB VBD VBG VBN VBP VBZ JJ JJR JJS RB RBR RBS`.

I then created the three levels: Binary, Probabilistic, and Pythagorean/Euclidean. I then computed the distance using the following distance metrics: City block, Cosine, Euclidean, Jaccard, and Jensen-Shannon. I did the clustering to compate seasons and the final table is as follows:

| Column       | Type    | Description                                                 |
|--------------|---------|-------------------------------------------------------------|
| doc_a     | Index   | First document formatted as (series, season)           |
| doc_b            | Index | Second document formatted as (series, season) |
| cityblock      | Feature | City block distance                                    |
| cosine            | Feature | Cosine distance                                     |
| euclidean            | Feature | Euclidean distance                                     |
| jaccard            | Feature | Jaccard distance                                     |
| js            | Feature | Jensen-Shannon distance                                     |

This file is located at `/final/clustering/ALL-PAIRS.csv`. This process is demonstrated in `TEXT_ANALYSIS.ipynb`

### 3.3 Principal Component Analysis (PCA)

The next model I created was for principal component analysis. For this model, I limited the VOCAB to nouns and plural nouns: `NN and NNS`. However, I noticed that in my corpus, some of the proper nouns were tagged as `NN`. To fix this, I did additional filtering on the VOCAB table to remove the names of the main and recurring characters of all 3 TV series. For the actual PCA model, I generated 10 components while also normalizing the input matrix. I also computed the principal components using episodes as the OHCO level due to memory constraints. There are two tables saved for PCA.

The first one is a table of documents and components:

| Column       | Type    | Description                                                 |
|--------------|---------|-------------------------------------------------------------|
|series_id|Index|ID of the series|
|season_id|Index| Season number|
|episode_id|Index|Episode number|
|PC0 to PC9 (10 columns)| Features | Associated values of the principal components|
|LIB-EPS details (multiple columns)| Features | Additional details joined from the ALL-LIB-EPS table|

This table is located at `/final/pca/ALL-DOC-COMPS.csv`

The second table is a table of components and word counts (i.e., the “loadings”):

| Column       | Type    | Description                                                 |
|--------------|---------|-------------------------------------------------------------|
|term_str|Index| String term|
|PC0 to PC9 (10 columns) | Features | Associated values of the principal components|
|n|Feature| Word count|

This table is located at `/final/pca/ALL-LOADINGS.csv`

To conduct further exploration, I also generated separate principal components for each separate TV series. The files are also located at `/final/pca`. This process is demonstrated in `TEXT_ANALYSIS.ipynb`

### 3.4 Topic Models (LDA)

For topic modelling, I also limited the VOCAB to nouns and plural nouns: `NN and NNS`. Similarly, I had to remove the names of main and recurring characters. I also noticed that there were some filler words that were not being tagged as stop words. To resolve this, I did additional filtering by removing the filler words before generating the topics. I used the following parameters for the topic model:
- Ngram range: [1, 2]
- Max features for the count vectorizer: 4000
- Number of components for LDA: 20
- Max iterations for LDA: 5
- Number of words used to characterize a topic: 7

After a few experiments in generating topics, I found that it is also effective to ignore terms that appear in more than 50% of the documents since a lot of the words in my corpus are usual conversational words. I did this by setting the *max_df* parameter of the CountVectorizer. I generated the topic models using scenes as the OHCO level. There are two tables saved for topic modelling.

The first table is a table of document and topic concentrations:

| Column       | Type    | Description                                                 |
|--------------|---------|-------------------------------------------------------------|
|series_id|Index|ID of the series|
|season_id|Index| Season number|
|episode_id|Index|Episode number|
|scene_id|Index|Scene number|
|T00 to T19 (20 columns)|Features|Topic concentration values|

This table is located at `/final/topic_modeling/ALL-DOCS.csv`

The second table is a table of topics and term counts:

**ADD HERE**

### 3.5 Word Embeddings (word2vec)

For the word embedding model, I limited the VOCAB to nouns and verbs: `'NN', 'NNS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'`. Similar to PCA and LDA, I filtered out the names of the main and recurring characters. I used the following parameters for the word2vec model:
- Window: 2
- Vector size: 256
- Minimum count: 50
- Workers: 4

The resulting table is a table of terms and embeddings:

| Column       | Type    | Description                                                 |
|--------------|---------|-------------------------------------------------------------|
|term_str|Index| String term|
|vector|Feature| word2vec vector|
|x|Feature| x coordinate|
|y|Feature| y coordinate|
|n|Feature| Term count|
|max_pos|Feature| Most frequent part-of-speech of the term|
|pos_group|Feature| Either Noun or Verb|

This table is located at `/final/word_embedding/ALL-COORDS.csv`. This process is demonstrated in `TEXT_ANALYSIS.ipynb`. For additional exploration, I also generated separate word2vec models for each TV series which are also located at `/final/word_embedding`.

### 3.6 Sentiment Analysis

For sentiment analysis, I made use of the SALEX NRC lexicon. I applied this to my VOCAB table where sentiment values were then associated to the terms. These sentiment values were also multiplied by the term TFIDF values. I computed the sentiment analysis using episodes as the OHCO level. I was able to generate two tables for this model.

The first table is a table of sentiment and emotion values as features of each term:

| Column       | Type    | Description                                                 |
|--------------|---------|-------------------------------------------------------------|
|term_str|Index| String term|
|anger|Feature| Anger value of the term|
|anticipation|Feature| Anticipation value of the term|
|disgust|Feature| Disgust value of the term|
|fear|Feature| Fear value of the term|
|joy|Feature| Joy value of the term|
|sadness|Feature| Sadness value of the term|
|surprise|Feature| Surprise value of the term|
|trust|Feature| Trust value of the term|
|sentiment|Feature| Sentiment value of the term|

This table is located at `/final/sentiment_analysis/ALL-VOCAB-SA.csv`.

The second table is a table of sentiment polarity and emotions for each document:

| Column       | Type    | Description                                                 |
|--------------|---------|-------------------------------------------------------------|
|series_id|Index|ID of the series|
|season_id|Index| Season number|
|episode_id|Index|Episode number|
|anger|Feature| Anger value of the document|
|anticipation|Feature| Anticipation value of the document|
|disgust|Feature| Disgust value of the document|
|fear|Feature| Fear value of the document|
|joy|Feature| Joy value of the document|
|sadness|Feature| Sadness value of the document|
|surprise|Feature| Surprise value of the document|
|trust|Feature| Trust value of the document|
|sentiment|Feature| Sentiment value of the document|

This table is located at `/final/sentiment_analysis/ALL-DOC-SA.csv`. This process is demonstrated at `TEXT_ANALYSIS.ipynb`